In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import csv 
import re
from langdetect import detect

In [14]:
f = open("homework.txt", "w")

for i in range(1,301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=" + str(i))
    soup = BeautifulSoup(page.content, features="lxml")
    links = soup.find_all('a', itemprop='url', class_='bookTitle')
    for link in links:
        fullLink = link.get('href')
        f.write('https://www.goodreads.com' + fullLink + '\n')
f.close()

In [ ]:
path = './storage/'


for i in range(1, 29701):
    
    folderName = "folder-" + str(i) + "/"
    fileName = "article_" + str(i) + ".html"
    
    f = open("homework3.txt", "r")
    url = f.readlines()[i-1]
    
    Path(path + folderName).mkdir(parents=True, exist_ok=True)

    page = requests.get(url)
    code = str(page.text)

    with open(path + folderName + fileName, "w", encoding="utf-8") as z:
        z.write(code)

    z.close()
    f.close()

In [25]:
data = ['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', "ratingCount",\
        "reviewCount", "plot", "numberOfPages", "publishingDate", "characters", "setting", "url"]

for i in range(19801, 19812): #fix with the index --> discuss in meet
    with open('./storage/folder-'+str(i)+"/article_"+str(i)+".html", 'rb') as html: 
        soup = BeautifulSoup(html,"html.parser")
    
    
    p = soup.find('div',id='description').contents[1].text

    if detect(p)=='en':
        
        lista=[]
        
        #title
        try:
            lista.append(soup.find('h1').text.strip())
        except:
            lista.append('')

        #bookseries
        try:
            lista.append(soup.find('h2',id='bookSeries').text.strip())
        except:
            lista.append('')

        #author name
        try:
            lista.append(soup.find('a',class_='authorName').text.strip())
        except:
            lista.append('')

        #rating value
        try:
            lista.append(soup.find('span', itemprop='ratingValue').text.strip())
        except:
            lista.append('')

        #ratingCount
        try:
            lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[0]\
                         .text.strip().replace('\r', '').replace('\n', '').split()[0])
        except:
            lista.append('')

        #reviewCount
        try:
            lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[1]\
                         .text.strip().replace('\r', '').replace('\n', '').split()[0])
        except:
            lista.append('')

        #plot
        try:
            plot = soup.find('div',id='description').text.strip()
            if plot[-7:] == '...more':
                lista.append(plot[:-7])
            else:
                lista.append(plot)
        except:
            lista.append('')

        #number of pages
        try:
            lista.append(soup.find('span', itemprop='numberOfPages').text.strip().split()[0])
        except:
            lista.append('')

        #Publishing Date
        try:
            a=soup.find_all('div', class_='row')[1].text
            match_obj = re.split('Published', re.split('by', a)[0])[1]
            lista.append(match_obj.strip())
        except:
            lista.append('')

        #characters
        try:
            lista.append(soup.find_all('div',class_="infoBoxRowItem")[4].text.strip()\
                         .replace('...more','').replace('...less',''))
        except:
            lista.append('')

        #setting
        try:
            lista.append(soup.find_all('div',class_="infoBoxRowItem")[5].text.strip()\
                         .replace('\n','').replace('\r',''))
        except:
            lista.append('')

        #URL
        lista.append(soup.find('link')['href'].strip())

        path="./storage/folder-"+str(i)+'/article_'+str(i)+'.tsv'
        with open(path, 'w', newline='') as f_output:
            tsv_output = csv.writer(f_output, delimiter='\t')
            tsv_output.writerow(data)
            tsv_output.writerow(lista)
            f_output.close()
